In [1]:
import bittensor
import torch
from tqdm import tqdm
from nuclei.gpt2 import GPT2Nucleus
from types import SimpleNamespace
from loguru import logger
bittensor.__debug_on__ = True
%load_ext autoreload

In [11]:
%autoreload 2

In [12]:

class Miner:
    def __init__( self, dataset: bittensor.Dataloader, endpoint: bittensor.Endpoint, child: bittensor.Endpoint ):
        
        # Local info.
        self.endpoint = endpoint
        
        # Child to call forward on.
        self.child = child
        
        # Text dataloader.
        self.dataset = dataset
        
        # Axon RPC server.
        # We attach the forward and backward passes to this miner class.
        # When this miner recieves a Forward/Backward request it calls these functions
        self.axon = bittensor.axon( local_port = self.endpoint.port )
        self.axon.attach_forward_function( self.forward )
        self.axon.attach_backward_function( self.backward )
        
        # Dendrite RPC Client.
        # Differentiable RPC function which calls Forward and Backward 
        # on passes endpoints.
        self.dendrite = bittensor.dendrite()
        
        # Torch NN Module with remote_forward and local_forwadd functions.
        # plus a routing function.
        self.nucleus = GPT2Nucleus()
        self.nucleus.attach_routing_function( self ) 
        
        # Base Torch optimizer.
        self.optimizer = torch.optim.AdamW(self.nucleus.parameters(), lr = 0.01, betas = (0.9, 0.95) )
                
    # Function is called by the nucleus to query child and get responses.
    def route( self, inputs: torch.int64, query: torch.float32 ) -> torch.float32:
        
        # Is this a leaf node.
        if self.child == None:
            response = [torch.zeros( [inputs.shape[0], inputs.shape[1], bittensor.__network_dim__ ])]
        
        # Otherwise, makes differentiable calls.
        else:
            # Takes a list of endpoints and a list of inputs
            # Sends inputs to endpoints.
            responses, return_codes = self.dendrite.forward_text (
                endpoints = [self.child], 
                x = [inputs] 
            )
            
        return responses[0]
    
    # Function which is called when this miner recieves a forward request from a dendrite.
    def forward ( self, pubkey:str, inputs: torch.float32, modality:int ) -> torch.FloatTensor:
        # Call nucleus (locally, i.e. using the distillation model instead of calling the child)
        # return the last hidden layer.  
        output = self.nucleus.local_forward (
            inputs = inputs        
        )
        return output.local_hidden

    # Function which is called when this miner recieves a backward request. (Off for now.)
    def backward ( self, pubkey:str, inputs_x:torch.float32, grads_dy:torch.float32, modality:int ) -> torch.FloatTensor:
        return None
    
    # Start the axon serving endpoint.
    def start(self):
        self.axon.start()
        
    # Tear down the axon serving endpoint.
    def __del__(self):
        self.axon.stop()

    # Run a single epoch.
    def epoch(self):
        # ---- Next Batch ----
        for iteration, inputs in enumerate(self.dataset.dataloader( 100 )):     
            
            # ---- Forward pass ----
            output = self.nucleus.remote_forward(
                inputs = inputs,
                training = True,
            )

            # ---- Backward pass ----
            output.loss = output.local_target_loss + output.distillation_loss + output.remote_target_loss
            output.loss.backward() # Accumulates gradients on the nucleus.
            self.optimizer.step() # Applies accumulated gradients.
            self.optimizer.zero_grad() # Zeros out gradients for next accummulation

        


In [13]:
# Dataset pulled from IPFS
dataset = bittensor.dataloader( max_corpus_size = 1000000 )

INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:149 - Retrieving a dataset file from the IPFS gateway...
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: naruda.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: self_reliance.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: pgp.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: animal_farm.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: prophet.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: magna_carta.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: animal_farm.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: bible_book_26.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: yoga_sutra.txt
INFO  

In [14]:
# Two fake bittensor endpoints for the miner.
endpoint_A = bittensor.endpoint( uid = 0, hotkey = '0', ip = '0.0.0.0', ip_type = 4, port = 8080 , modality = 0, coldkey = 'N/A'  )
endpoint_B = bittensor.endpoint( uid = 1, hotkey = '1', ip = '0.0.0.0', ip_type = 4, port = 8081 , modality = 0, coldkey = 'N/A'  )

In [19]:
# Create miner A
if "miner_A" in locals():
    del miner_A
miner_A = Miner( dataset = dataset, endpoint = endpoint_A, child = endpoint_B )
miner_A.start()

SUCCESS |bittensor._axon.axon_impl:_serve:465 - Axon is serving on: 127.0.0.1:8080


In [20]:
# Create miner B
if "miner_B" in locals():
    del miner_B
miner_B = Miner( dataset = dataset, endpoint = endpoint_B, child = None )
miner_B.start()

SUCCESS |bittensor._axon.axon_impl:_serve:465 - Axon is serving on: 127.0.0.1:8081


In [21]:
# Start training miner A.
miner_A.epoch()

DEBUG   |bittensor._dendrite.dendrite_impl:_get_or_create_receptor_for_endpoint:266 - Create receptor for endpoint: <endpoint uid: 1 hotkey: 1 ip: /ipv4/0.0.0.0:8081 modality: 0 coldkey: N/A>
SUCCESS |bittensor._wallet.wallet_impl:_load_hotkey:242 - Loaded hotkey: 0x76a02b5371044ee1c7d031e5018e872acfcf6d3673f346a030e9bb1e167e621b
DEBUG   |bittensor._receptor.receptor_impl:forward:246 - Dendrite Forward Request ---> to:/ipv4/0.0.0.0:8081, inputs:torch.Size([10, 20]), mode:0
<bound method Miner.forward of <__main__.Miner object at 0x143279670>>
DEBUG   |bittensor._axon.axon_impl:Forward:88 - -> Got Forward request: 0x76a02b5371044ee1c7d031e5018e872acfcf6d3673f346a030e9bb1e167e621b, size:96
DEBUG   |bittensor._axon.axon_impl:_forward:314 - Axon Forward Request ---> from:0x76a02b5371044ee1c7d031e5018e872acfcf6d3673f346a030e9bb1e167e621b, inputs:torch.Size([10, 20])
DEBUG   |bittensor._axon.axon_impl:_forward:332 - Axon Forward Response <--- to:0x76a02b5371044ee1c7d031e5018e872acfcf6d3673f3

KeyboardInterrupt: 